In [3]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import timeit
import statistics
import time
import torch
# import torch_geometric
import importlib

from data_utils import synthetic_data
# from data_utils import graph_constructors
from data_utils import group_to_image_constructors

import networkx as nx

import importlib

# from Unet import helper
# from Unet import simulation

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

In [2]:
# from collections import defaultdict
# import torch.nn.functional as F
# from Unet.loss import dice_loss

# def calc_loss(pred, target, metrics, bce_weight=0.5):
#     bce = F.binary_cross_entropy_with_logits(pred, target)

#     pred = F.sigmoid(pred)
#     dice = dice_loss(pred, target)

#     loss = bce * bce_weight + dice * (1 - bce_weight)

#     metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
#     metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
#     metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

In [5]:
def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    

    intersection = (pred * target)
    
    loss = (1 - ((2. * intersection + smooth) / (pred + target + smooth)))
    
    return loss.mean()

In [8]:
pred = torch.tensor([0.5, 0.5])
target = torch.tensor([1,0])

dice_loss(pred, target)

tensor(0.2667)

In [4]:
target = torch.tensor(np.random.randint(0,2, size = (4,2,224,224)))

In [5]:
pred = torch.tensor(np.random.uniform(size = (4,2,224,224)))

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

pred.to(device)

tensor([[[[0.6756, 0.2125, 0.5895,  ..., 0.9198, 0.3196, 0.9708],
          [0.0282, 0.1970, 0.8427,  ..., 0.3869, 0.5408, 0.8237],
          [0.1669, 0.9908, 0.1743,  ..., 0.3190, 0.4256, 0.0457],
          ...,
          [0.1062, 0.0429, 0.6247,  ..., 0.8422, 0.8561, 0.0882],
          [0.2740, 0.5896, 0.0653,  ..., 0.7692, 0.2280, 0.5653],
          [0.6571, 0.7950, 0.8686,  ..., 0.9004, 0.2801, 0.6024]],

         [[0.5923, 0.1424, 0.9117,  ..., 0.1168, 0.4346, 0.9234],
          [0.5138, 0.4940, 0.6516,  ..., 0.6866, 0.4886, 0.2681],
          [0.7835, 0.9696, 0.4186,  ..., 0.4028, 0.9711, 0.6176],
          ...,
          [0.5595, 0.7075, 0.3274,  ..., 0.8960, 0.0926, 0.6693],
          [0.4136, 0.0322, 0.5911,  ..., 0.4943, 0.5439, 0.8181],
          [0.7175, 0.5903, 0.4602,  ..., 0.0547, 0.5987, 0.9855]]],


        [[[0.0546, 0.2808, 0.0985,  ..., 0.6476, 0.5078, 0.2566],
          [0.8905, 0.9493, 0.5821,  ..., 1.0000, 0.9242, 0.5654],
          [0.9745, 0.9318, 0.8083,  ...,

In [18]:
def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    

    print('pred.shape: ', pred.shape, ', target.shape: ', target.shape)

    intersection = (pred * target).sum(dim=2).sum(dim=2)

    print('intersection.shape: ', intersection.shape)
    
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    
    return loss.mean()

In [19]:
dice_loss(pred, target, smooth = 1.)

pred.shape:  torch.Size([4, 2, 224, 224]) , target.shape:  torch.Size([4, 2, 224, 224])
intersection.shape:  torch.Size([4, 2])


tensor(0.4999, dtype=torch.float64)

In [29]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 3, 1]
labels_pred = [0, 0, 0, 1, 2, 1]

print(metrics.adjusted_rand_score(labels_true, labels_pred))
print(metrics.normalized_mutual_info_score(labels_true, labels_pred))
print(metrics.homogeneity_completeness_v_measure(labels_true, labels_pred))

1.0


C:\Users\andy.knapper.BIZFITECH\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


1.0
(1.0, 1.0, 1.0)
